In [1]:
import digits
import tensorflow as tf
from time import time
import itertools as it
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
DATA_PATH = 'digits.csv'

LEARNING_RATE = 0.1
BATCH_SIZE = 128
EPOCHS = 30
MODEL_DIR = 'cnn-res-da/model_{}'

In [3]:
labels, pixels = digits.read(DATA_PATH)
pixels_sq = pixels.reshape(-1, 28, 28)

In [4]:
train_label, test_label, train_pixel, test_pixel = train_test_split(labels, pixels_sq, test_size=0.1, 
                                                                    stratify=labels, shuffle=True, random_state=0)
train_label, dev_label, train_pixel, dev_pixel = train_test_split(train_label, train_pixel, test_size=0.11111111, 
                                                                  stratify=train_label, shuffle=True, random_state=0)

In [5]:
train_label.shape, dev_label.shape, test_label.shape

((8000,), (1000,), (1000,))

In [6]:
train_pixel.shape, dev_pixel.shape, test_pixel.shape

((8000, 28, 28), (1000, 28, 28), (1000, 28, 28))

In [7]:
num_labels = len(set(labels))
num_labels

10

In [8]:
def model_fn(features, labels, mode):
    c1 = tf.layers.conv2d(features, 4, 3, activation=tf.nn.relu) # 26 x 26
    c2 = tf.layers.conv2d(c1, 4, 3, activation=tf.nn.relu) # 24 x 24
    m1 = tf.layers.max_pooling2d(c2, 2, 2) # 12 x 12
    c3 = tf.layers.conv2d(m1, 8, 3, activation=tf.nn.relu) # 10 x 10
    c4 = tf.layers.conv2d(c3, 8, 3, activation=tf.nn.relu) # 8 x 8
    c5 = tf.layers.conv2d(c4, 8, 3, activation=tf.nn.relu) # 6 x 6
    m2 = tf.layers.max_pooling2d(c5, 2, 2) # 3 x 3
    fc = tf.concat([tf.layers.flatten(m1),
                    tf.layers.flatten(m2)],
                   axis=-1)
    logits = tf.layers.dense(fc, num_labels)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        proba = tf.nn.softmax(logits, axis=-1)
        predictions = {
            'logits': logits,
            'probabilities': proba,
            'labels': tf.argmax(proba, axis=-1),
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.softmax_cross_entropy(labels, logits)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss)
    
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

In [9]:
import math

def augment_data(p, lbl, rotations):
    def rotate_processing(angle, p, lbl):
        if angle != 0:
            prot = tf.expand_dims(p, axis=-1)
            prot = tf.contrib.image.rotate(prot, angle / 180 * math.pi)
            prot = tf.squeeze(prot, axis=-1)
        else:
            prot = p
        return prot, lbl
    
    all_data = [rotate_processing(r, p, lbl) for r in rotations]
    all_pixels, all_labels = zip(*all_data)
    all_pixels = tf.concat(all_pixels, axis=0)
    all_labels = tf.concat(all_labels, axis=0)
    return all_pixels, all_labels

In [10]:
def td_input_fn(pixels, labels, batch_size, epochs, is_training, buffer=100):
    if is_training:
        pixels, labels = augment_data(pixels, labels, [-10, -5, 0, 5, 10])
    
    ds = tf.data.Dataset.from_tensor_slices((pixels, labels))
    
    def preprocessing(p, lbl):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        lbl_1h = tf.one_hot(lbl, num_labels, dtype=tf.float32)
        return p3d, lbl_1h
    
    ds = ds.map(preprocessing, num_parallel_calls=8)
    
    if is_training:
        ds = ds.shuffle(buffer * batch_size)
        
    ds = ds.batch(batch_size)
    ds = ds.repeat(epochs)
    ds = ds.prefetch(buffer)
    
    return ds

In [11]:
def pred_input_fn(pixels, batch_size, buffer=100):
    ds = tf.data.Dataset.from_tensor_slices(pixels)
    
    def preprocessing(p):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        return p3d
        
    ds = ds.map(preprocessing, num_parallel_calls=8)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer)
    
    return ds

In [12]:
model_dir = MODEL_DIR.format(int(time()))
config = tf.estimator.RunConfig(tf_random_seed=0,
                                save_summary_steps=50,
                                save_checkpoints_steps=250,
                                keep_checkpoint_max=None)

clf = tf.estimator.Estimator(model_fn, model_dir, config)

INFO:tensorflow:Using config: {'_model_dir': 'cnn-res-da/model_1540253161', '_tf_random_seed': 0, '_save_summary_steps': 50, '_save_checkpoints_steps': 250, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FC19D814E0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
train_input_fn = lambda: td_input_fn(train_pixel, train_label, BATCH_SIZE, EPOCHS, True)
dev_input_fn = lambda: td_input_fn(dev_pixel, dev_label, BATCH_SIZE, 1, False)

train_spec = tf.estimator.TrainSpec(train_input_fn)
eval_spec = tf.estimator.EvalSpec(dev_input_fn, None, start_delay_secs=0.1, throttle_secs=0.1)

In [14]:
tf.estimator.train_and_evaluate(clf, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 250 or save_checkpoints_secs None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into cnn-res-da/model_1540253161\model.ckpt.
INFO:tensorflow:cnn-res-da/model_1540253161\model.ckpt-0 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:loss = 2.5741704, step = 0
INFO:tensorflow:global_step/sec: 65.9975
INFO:tensorflow:loss = 0.2615968, step = 100 (1.515 sec)
INFO:tensorflow:global_step/sec: 130.633
INFO:tensorflow:loss = 0.21092439, step = 200 (0.766 sec)
INFO:tensorflow:Saving check

INFO:tensorflow:global_step/sec: 125.177
INFO:tensorflow:loss = 0.04724333, step = 1900 (0.799 sec)
INFO:tensorflow:Saving checkpoints for 2000 into cnn-res-da/model_1540253161\model.ckpt.
INFO:tensorflow:cnn-res-da/model_1540253161\model.ckpt-2000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:07:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res-da/model_1540253161\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:07:28
INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.08558971
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: cnn-res-da/model_1540253161\model.ckpt-2000
INFO:tensorflow:global_step/sec: 35.8397
INFO:tensorflow:loss = 0.062337615, step = 2000 (2.7

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3750: cnn-res-da/model_1540253161\model.ckpt-3750
INFO:tensorflow:global_step/sec: 40.0094
INFO:tensorflow:loss = 0.0031716395, step = 3800 (2.499 sec)
INFO:tensorflow:global_step/sec: 130.643
INFO:tensorflow:loss = 0.008360917, step = 3900 (0.750 sec)
INFO:tensorflow:Saving checkpoints for 4000 into cnn-res-da/model_1540253161\model.ckpt.
INFO:tensorflow:cnn-res-da/model_1540253161\model.ckpt-4000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:08:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res-da/model_1540253161\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:08:03
INFO:tensorflow:Saving dict for global step 4000: global_step = 4000, loss = 0

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:08:31
INFO:tensorflow:Saving dict for global step 5750: global_step = 5750, loss = 0.10082442
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5750: cnn-res-da/model_1540253161\model.ckpt-5750
INFO:tensorflow:global_step/sec: 38.7615
INFO:tensorflow:loss = 0.0011528722, step = 5800 (2.580 sec)
INFO:tensorflow:global_step/sec: 113.82
INFO:tensorflow:loss = 0.00028375827, step = 5900 (0.879 sec)
INFO:tensorflow:Saving checkpoints for 6000 into cnn-res-da/model_1540253161\model.ckpt.
INFO:tensorflow:cnn-res-da/model_1540253161\model.ckpt-6000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:08:35
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res-da/model_1540253161\model

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:09:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res-da/model_1540253161\model.ckpt-7750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:09:03
INFO:tensorflow:Saving dict for global step 7750: global_step = 7750, loss = 0.109772846
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7750: cnn-res-da/model_1540253161\model.ckpt-7750
INFO:tensorflow:global_step/sec: 33.313
INFO:tensorflow:loss = 0.0012221002, step = 7800 (3.002 sec)
INFO:tensorflow:global_step/sec: 125.513
INFO:tensorflow:loss = 0.00017278794, step = 7900 (0.797 sec)
INFO:tensorflow:Saving checkpoints for 8000 into cnn-res-da/model_1540253161\model.ckpt.
INFO:tensorflow:cnn-res-da/model_1540253161\model.ckpt-8000 is not in all_model_checkpoint_paths. Manually

({'loss': 0.118591264, 'global_step': 9390}, [])

In [15]:
checkpoint_path = r'cnn-res-da/model_1540253161\model.ckpt-4750'

In [16]:
test_input_fn = lambda: pred_input_fn(test_pixel, BATCH_SIZE)

test_pred = clf.predict(test_input_fn, predict_keys=['labels'], checkpoint_path=checkpoint_path)
test_pred = [p['labels'] for p in test_pred]
test_pred = np.asarray(test_pred)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res-da/model_1540253161\model.ckpt-4750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [17]:
eval_input_fn = lambda: td_input_fn(test_pixel, test_label, BATCH_SIZE, 1, False)
clf.evaluate(eval_input_fn, checkpoint_path=checkpoint_path)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:11:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-res-da/model_1540253161\model.ckpt-4750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:11:07
INFO:tensorflow:Saving dict for global step 4750: global_step = 4750, loss = 0.091699034
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4750: cnn-res-da/model_1540253161\model.ckpt-4750


{'loss': 0.091699034, 'global_step': 4750}

In [18]:
print(classification_report(test_label, test_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98        99
           1       1.00      0.98      0.99       110
           2       0.98      0.95      0.97       104
           3       0.97      0.98      0.98       101
           4       0.96      0.99      0.97        97
           5       0.99      0.98      0.98        91
           6       0.98      1.00      0.99       100
           7       0.96      0.98      0.97       104
           8       0.96      0.94      0.95        95
           9       0.98      0.99      0.98        99

   micro avg       0.98      0.98      0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000



In [19]:
cm = confusion_matrix(test_label, test_pred)
cm

array([[ 97,   0,   0,   0,   0,   1,   1,   0,   0,   0],
       [  0, 108,   0,   0,   0,   0,   0,   0,   2,   0],
       [  0,   0,  99,   0,   1,   0,   0,   3,   1,   0],
       [  0,   0,   0,  99,   0,   0,   0,   0,   1,   1],
       [  0,   0,   0,   0,  96,   0,   0,   1,   0,   0],
       [  0,   0,   0,   1,   0,  89,   0,   0,   0,   1],
       [  0,   0,   0,   0,   0,   0, 100,   0,   0,   0],
       [  0,   0,   2,   0,   0,   0,   0, 102,   0,   0],
       [  1,   0,   0,   2,   2,   0,   1,   0,  89,   0],
       [  0,   0,   0,   0,   1,   0,   0,   0,   0,  98]], dtype=int64)